# Scrapy + Scraper API

In [1]:
pip install scrapy

     |████████████████████████████████| 245kB 4.3MB/s 
     |████████████████████████████████| 2.6MB 17.4MB/s 
     |████████████████████████████████| 3.1MB 45.0MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 3.2MB 52.8MB/s 
     |████████████████████████████████| 245kB 41.6MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 61kB 3.6MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=14d29110d846ae57da56f293d1f8930b6f8443b3cd003ac7c4df6209d2ffcff0
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=b20ffad86d9c4d37e323a9b85ca84aafbef14f4176622cbcbf6d0940fcb2d1ed
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatch

In [2]:
import scrapy
from urllib.parse import urlencode
from urllib.parse import urlparse
import json
from datetime import datetime
API_KEY = '6e2f13665a14d50e37acdfc9636b4877'

In [50]:
def get_url(url):
   payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr'}
   proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
   return proxy_url

def create_google_url(query, site=''):
   google_dict = {'q': query, 'num': 100, }
   if site:
       web = urlparse(site).netloc
       google_dict['as_sitesearch'] = web
       return 'http://www.google.com/search?' + urlencode(google_dict)
   return 'http://www.google.com/search?' + urlencode(google_dict)

queries = ['innovation', 'gestion']
class GoogleSpider(scrapy.Spider):
   name = 'google'
   allowed_domains = ['www.google.com']
   custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}
    
   def start_requests(self, queries):
     for query in queries:
      url = create_google_url(query)
      yield scrapy.Request(get_url(url), callback=self.parse, meta={'pos': 0}) 

   def parse(self, response):
       di = json.loads(response.text)
       pos = response.meta['pos']
       dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
       for result in di['organic_results']:
           title = result['title']
           snippet = result['snippet']
           link = result['link']
           item = {'title': title, 'snippet': snippet, 'link': link, 'position': pos, 'date': dt}
           pos += 1
           yield item
       next_page = di['pagination']['nextPageUrl']
       if next_page:
           yield print(scrapy.Request(get_url(next_page), callback=self.parse, meta={'pos': pos}).text)

In [51]:
if __name__ == '__main__':
    result = list(GoogleSpider().start_requests(queries))

In [52]:
result

[<GET http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr>,
 <GET http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dgestion%26num%3D100&autoparse=true&country_code=fr>]

In [56]:
link = []
for i in range(len(result)):
    link = link + [str(result[i])[5:-1]]
print(link)

['http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr', 'http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dgestion%26num%3D100&autoparse=true&country_code=fr']


In [78]:
# On importe la fonction 'get' (téléchargement) de 'requests' 
# Et la classe 'Selector' (Parsing) de 'scrapy'
from requests import get
from scrapy import Selector
# Lien de la page à scraper
url = link
response = []
for i in range(len(url)):
  response = response + [get(url[i])]
  source = [] # Le code source de la page 
  while response[i].status_code == 200 and i >= 0 :
    # Si la requete s'est bien passee
    source = source + [response[i].text]
    i = i - 1

In [80]:
source

['{"search_information":{"total_results":895,"time_taken_displayed":0,"query_displayed":"gestion"},"ads":[],"knowledge_graph":{"title":"","description":"","source":{"name":""},"related":[],"related_link":null,"social_media":[],"see_more_about":[{"title":"Recherches associées","link":"https://www.google.com/search?num=100&gl=FR&q=Management&stick=H4sIAAAAAAAAAONgFuLQz9U3MIkvKVOCs7SEs5Ot9JPzc3Pz86yKM1NSyxMrixexcvkm5iWmp-am5pXsYGUEAE6MeIw7AAAA&sa=X&ved=2ahUKEwiYn9eRwIfuAhURip4KHU2ODzMQMSgAMHh6BAgVEAE"},{"title":"Voir d\'autres éléments (plus de\xa010)","link":"https://www.google.com/search?num=100&gl=FR&q=Management&stick=H4sIAAAAAAAAAONgFuLQz9U3MIkvKVOCs7SEs5Ot9JPzc3Pz86yKM1NSyxMrixexcvkm5iWmp-am5pXsYGUEAE6MeIw7AAAA&sa=X&ved=2ahUKEwiYn9eRwIfuAhURip4KHU2ODzMQzTooATB4egQIFRAC"},{"title":"Leadership","link":"https://www.google.com/search?num=100&gl=FR&q=Leadership&stick=H4sIAAAAAAAAAONgFuLQz9U3MIkvKVPiArGMTItKjIq1-Jzzc3Pz84IzU1LLEyuLF7Fy-aQmpqQWFWdkFuxgZQQArzrTzjgAAAA&sa=X&ved=2ahUKEwiYn9eR

In [116]:
import re

my_links = []
pattern = re.compile("\"link\"[^,]+")
my_links = my_links + list(set([x for x in pattern.findall(str(source))]))
my_links = [i[8:] for i in my_links]
my_links = [i[:-1] for i in my_links]

mylinks=[]
for i in my_links:
    if "https://www.google" not in i:
        mylinks.append(i)

mylinks

['https://www.bpifrance.fr/',
 'https://arcinnovation.fr/',
 'https://www.cairn.info/revue-innovations.htm',
 'https://fr.planisware.com/hub/blog/le-principe-de-la-gestion-dun-projet',
 'https://solidarites-sante.gouv.fr/systeme-de-sante-et-medico-social/recherche-et-innovation/forfait-innovation',
 'https://mon-expert-en-gestion.fr/',
 'https://espaceclientpro.orange.fr/',
 'https://www.linternaute.fr/dictionnaire/fr/definition/gestion/',
 'https://www.dictionary.com/browse/innovation',
 'https://www.industrie-techno.com/innovation',
 'https://dictionnaire.reverso.net/francais-definition/gestion',
 'https://www.france-gestion.fr/',
 'https://www.ouest-france.fr/economie/innovation-le-dynamisme-des-entreprises-de-l-ouest-en-faveur-du-climat-trouve-du-soutien-7107329',
 'https://www.expensya.com/fr/',
 'https://1mot.net/gestion',
 'https://www.legalplace.fr/guides/gestion-entreprise/',
 'https://www.linguee.fr/francais-anglais/traduction/gestion.html',
 'https://www.gererseul.com/',
 'h